In [12]:
import pandas as pd

# Try reading the file with different encodings
try:
    # Try UTF-8 first (default)
    data = pd.read_csv('SST_Hoffman_Harmonized_AC_ES.csv')
except UnicodeDecodeError:
    try:
        # Try ISO-8859-1 (common for Western European languages)
        data = pd.read_csv('SST_Hoffman_Harmonized_AC_ES.csv', encoding='ISO-8859-1')
    except UnicodeDecodeError:
        try:
            # Try latin1 (similar to ISO-8859-1)
            data = pd.read_csv('SST_Hoffman_Harmonized_AC_ES.csv', encoding='latin1')
        except UnicodeDecodeError:
            # Try windows-1252 (common for Windows systems)
            data = pd.read_csv('SST_Hoffman_Harmonized_AC_ES.csv', encoding='windows-1252')

# Display the first few rows to verify
print(data.head())

ParserError: Error tokenizing data. C error: Expected 2 fields in line 10, saw 3


In [5]:
import numpy as np
import pandas as pd
import pymc as pm
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# Load the Excel file
data = pd.read_excel('SST_Hoffman_Harmonized_AC_ES.csv')

# Display the first few rows to verify
print(data.head())

# Extract relevant columns
longitude = data['Longitude'].values
latitude = data['Latitude'].values

# Select a time slice (e.g., 130,000 years ago)
time_slice = '130000'
sst = data[time_slice].values

# Handle missing or problematic data (e.g., NaNs or red-highlighted values)
# Replace NaNs with the mean SST value for simplicity
sst = np.where(np.isnan(sst), np.nanmean(sst), sst)

# Define the Gaussian Process model
def gp_model(longitude, latitude, sst):
    # Define the covariance function (squared exponential kernel)
    def cov_func(d, amplitude, length_scale):
        return amplitude**2 * np.exp(-0.5 * (d / length_scale)**2)

    # Priors for the GP parameters
    amplitude = pm.Uniform('amplitude', 0, 10)
    length_scale = pm.Uniform('length_scale', 0, 10)
    noise = pm.Uniform('noise', 0, 1)

    # Calculate the pairwise distances between points
    coords = np.vstack([longitude, latitude]).T
    D = cdist(coords, coords)

    # Covariance matrix
    cov = cov_func(D, amplitude, length_scale) + noise**2 * np.eye(len(sst))

    # GP likelihood
    observed = pm.MvNormal('observed', mu=np.zeros(len(sst)), cov=cov, observed=sst)

    return locals()

# Create the PyMC model
model = pm.Model(gp_model(longitude, latitude, sst))

# Perform MCMC sampling
with model:
    trace = pm.sample(1000, tune=1000, cores=2)

# Generate a high-resolution grid for interpolation
grid_longitude = np.linspace(min(longitude), max(longitude), 100)
grid_latitude = np.linspace(min(latitude), max(latitude), 100)
grid_longitude, grid_latitude = np.meshgrid(grid_longitude, grid_latitude)
grid_coords = np.vstack([grid_longitude.ravel(), grid_latitude.ravel()]).T

# Interpolate SST values on the grid
def interpolate_sst(trace, grid_coords, longitude, latitude, sst):
    # Extract posterior samples
    amplitude_samples = trace['amplitude']
    length_scale_samples = trace['length_scale']
    noise_samples = trace['noise']

    # Calculate the pairwise distances between grid points and data points
    D_grid = cdist(grid_coords, np.vstack([longitude, latitude]).T)

    # Initialize the interpolated SST array
    sst_interp = np.zeros((len(grid_coords), len(amplitude_samples)))

    # Interpolate SST for each posterior sample
    for i in range(len(amplitude_samples)):
        cov_grid = cov_func(D_grid, amplitude_samples[i], length_scale_samples[i])
        cov_data = cov_func(cdist(np.vstack([longitude, latitude]).T, np.vstack([longitude, latitude]).T),
                            amplitude_samples[i], length_scale_samples[i]) + noise_samples[i]**2 * np.eye(len(sst))
        K_inv = np.linalg.inv(cov_data)
        sst_interp[:, i] = cov_grid @ K_inv @ sst

    # Return the mean interpolated SST
    return sst_interp.mean(axis=1)

# Perform interpolation
sst_interp = interpolate_sst(trace, grid_coords, longitude, latitude, sst)

# Reshape the interpolated SST to the grid shape
sst_interp_grid = sst_interp.reshape(grid_longitude.shape)

# Plot the interpolated SST map
plt.figure(figsize=(10, 8))
plt.contourf(grid_longitude, grid_latitude, sst_interp_grid, levels=50, cmap='viridis')
plt.colorbar(label='Sea Surface Temperature (SST)')
plt.scatter(longitude, latitude, c=sst, edgecolors='black', label='Data Points')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title(f'High-Resolution Interpolated SST Map ({time_slice} years ago)')
plt.legend()
plt.show()

NameError: name 'pd' is not defined

NameError: name 'pd' is not defined